This is a collection of methods to aggregate some key household level data at the neighbourhood level.  
It relies on the geopoints being included in the export (thanks fam) and makes use of the Neighbourhoods class to label the households and then uses the household labels to aggregate statistics about useage to the neighbourhood level.

It is still a work in progress.

In [ ]:
from kw_neighbourhoods import Neighbourhoods # borrowed from December_Ops collection
import pandas as pd

In [ ]:
tdb = r'source_files/City of Waterloo and Kitchener Planning district Geometry.json'
# opens a tinydb of geometries and provides a method to see if arbitrary geopoints are within one of the neighbourhoods 
# accessible by the object
kw = Neighbourhoods(tdb)
kw.extract_shapes() # takes geometries and organizes them in the .nhood_shapes attribute

In [ ]:
%ls

In [ ]:
data_types = {'Latitude' : 'float',
              'Longitude' : 'float'
              }


In [ ]:
cch = ['Visit Date','Client ID','Client Age','Client Gender','City','Latitude','Longitude','Household ID','Household Size',
'Household Primary Income Source','Program Name','HH Mem 1- ID','HH Mem 1- Age','HH Mem 1- Gender',
'HH Mem 2- ID','HH Mem 2- Age','HH Mem 2- Gender','HH Mem 3- ID','HH Mem 3- Age','HH Mem 3- Gender',
'HH Mem 4- ID','HH Mem 4- Age','HH Mem 4- Gender','HH Mem 5- ID','HH Mem 5- Age','HH Mem 5- Gender','HH Mem 6- ID',
'HH Mem 6- Age','HH Mem 6- Gender','HH Mem 7- ID','HH Mem 7- Age','HH Mem 7- Gender','HH Mem 8- ID','HH Mem 8- Age',
'HH Mem 8- Gender','HH Mem 9- ID','HH Mem 9- Age','HH Mem 9- Gender','HH Mem 10- ID','HH Mem 10- Age','HH Mem 10- Gender',
'HH Mem 11- ID','HH Mem 11- Age','HH Mem 11- Gender','HH Mem 12- ID','HH Mem 12- Age','HH Mem 12- Gender',
'HH Mem 13- ID','HH Mem 13- Age','HH Mem 13- Gender','HH Mem 14- ID','HH Mem 14- Age','HH Mem 14- Gender',
'HH Mem 15- ID','HH Mem 15- Age','HH Mem 15- Gender']

In [ ]:
fhh =['Visit Date','Client ID','Client Age','Client Gender','City','Latitude','Longitude','Household ID',
      'Household Size','Household Primary Income Source','HH Mem 1- ID','HH Mem 1- Age','HH Mem 1- Gender',
'HH Mem 2- ID','HH Mem 2- Age','HH Mem 2- Gender','HH Mem 3- ID','HH Mem 3- Age','HH Mem 3- Gender',
'HH Mem 4- ID','HH Mem 4- Age','HH Mem 4- Gender','HH Mem 5- ID','HH Mem 5- Age','HH Mem 5- Gender','HH Mem 6- ID',
'HH Mem 6- Age','HH Mem 6- Gender','HH Mem 7- ID','HH Mem 7- Age','HH Mem 7- Gender','HH Mem 8- ID','HH Mem 8- Age',
'HH Mem 8- Gender','HH Mem 9- ID','HH Mem 9- Age','HH Mem 9- Gender','HH Mem 10- ID','HH Mem 10- Age','HH Mem 10- Gender',
'HH Mem 11- ID','HH Mem 11- Age','HH Mem 11- Gender','HH Mem 12- ID','HH Mem 12- Age','HH Mem 12- Gender',
'HH Mem 13- ID','HH Mem 13- Age','HH Mem 13- Gender','HH Mem 14- ID','HH Mem 14- Age','HH Mem 14- Gender',
'HH Mem 15- ID','HH Mem 15- Age','HH Mem 15- Gender']

In [ ]:
# open the file of source data: visits to the programs and other information exported from l2f
fh_path = 'source_files/EFHP_full_year_caseload_2018.csv' # EFHP caseload
cc_path = 'source_files/cc2018.csv' # community programs
data1 = pd.read_csv(fh_path, usecols=fhh, dtype=data_types, low_memory=False) # EFHP
data2 = pd.read_csv(cc_path, usecols=cch, dtype=data_types, low_memory=False)
data1['Program Name'] = 'EFHP' # program name column is missing, so add it with EFHP as the value
data = pd.concat([data1,data2], axis =0, ignore_index=True, sort=False) # combine both datasets into one


In [ ]:
data.columns.values

In [ ]:
source_file = 'hof_2018_food_programs' # string to prepend to output files

In [ ]:
def find_neighbourhood(row):
    '''
    uses the Neighbourhoods class to extract the neighbourhood
    or 'Unknown' if the lat and long are not in the csv
    can be used in a lambda function to through .apply method to
    create a new column with a neighbourhood value in it
    '''
    lat_lng = (row['Latitude'], row['Longitude'])
    if all(lat_lng):
        return kw.find_in_shapes(*lat_lng)
    else:
        return 'Unknown'

In [ ]:
# Add a column that contains the neighbourhood based off the values in the Latitude and Longitude column
data['Neighbourhood'] = data.apply(lambda row: find_neighbourhood(row), axis=1)

In [ ]:
# write out a new version of the source csv with a neighbourhood column
data.to_csv('~/datascience/mapping_notebooks/output_files/{}_with_neighbourhoods.csv'.format(source_file))

In [ ]:
# what are the neighbourhoods that Households have visited.
# note: False, is passed through by kw_neighbourhoods as the HH is not in KW, or no valid [lat, lng] was available
# for them in the dataset.
data['Neighbourhood'].unique()

In [ ]:
len(kw.nhood_shapes)

In [ ]:
# have a look to see if things worked properly
data['Neighbourhood'].head()

In [ ]:
# drop the records that do not contain a valid neighbourhood
data = data[data.Neighbourhood != False]

In [ ]:
data['Neighbourhood'].unique()

In [ ]:
# lets cleanup the source of income fields and aggregate some of the values where it makes sense
# first, what values are there?
data['Household Primary Income Source'].unique()

In [ ]:
# change nan values to 'unknown'
data['Household Primary Income Source'].fillna('Unknown or None', inplace=True)

In [ ]:
# streamline the SOI field with a mapping of five categories
prov = 'Provincial'
fed = 'Federal'
other = 'Other'
no_un = 'Unknown or None'
emp = 'Partial or full time employment'

soi_map = {
    'Ontario Works (OW)': prov, 
    'No Income': no_un,
    'Ontario Disability Support Program (ODSP)': prov,
    'Canadian Pension Plan (CPP)': fed, 
    'Private Disability': other, 
    'Immigration': fed,
    'Employed: Full-Time': emp, 
    'Employed: Part-Time': emp, 
    'Child Support': other,
    'Old Age Security (OAS)': fed, 
    'Employment Insurance (EI)': fed,
    'Other (Specify)': other,
    'Child Tax Benefit': fed,
    'Self Employed': emp,
    'Student Loans': other,
    'Temporary Work': emp,
    'WSIB': prov,
    'Universal Child Benefit': fed,
    "Children's Aid": other,
    'Private Pension': other,
    'Family Support': other,
    'Spouse/Family Support': other,
    'Scholarships': other,
    'provincial_disability': prov,
    'Unknown': no_un
}

# this will change values to what we prefer
data['Household Primary Income Source'] = data['Household Primary Income Source'].replace(soi_map)

In [ ]:
data['Household Primary Income Source'].unique()

In [ ]:
# group the data by households and pull out the first occurance of each unique 'Household ID'
# https://stackoverflow.com/questions/20067636/pandas-dataframe-get-first-row-of-each-group
households = data.groupby('Household ID').first()

In [ ]:
# count the number of occurances of the Household ID - a way to count household visits for some service
# (due to the way that L2F deals with household members moving between households, there is likely some double counting involved)
# Insert the count as a new column
households['Number of Visits'] = data['Household ID'].value_counts()

In [ ]:
# reset the index and now we have a dataset with Household data aggregated a bit, with a number of visits and neighbourhood data
households.reset_index(inplace=True)
households.head()

In [ ]:
# how many unique HH id's?
households['Household ID'].nunique()

In [ ]:
# now groupby Neighbourhood so we can start generating some Neighbourhood level stats
# especially HH size, visits and income source
nhood_data = households.groupby(['Neighbourhood', 'Household Primary Income Source'])

In [ ]:
# what functions do we want to apply to each column to get some basic stats?
agg_functions = {'Household ID': 'count', 'Household Size': ['mean', 'sum'], 'Number of Visits': ['mean', 'sum']}

In [ ]:
stats_nhoods = nhood_data.agg(agg_functions)

In [ ]:
# sort the neighbourhoods by neighbourhood name, and then visit count starting with the highest
sorted_nhoods = stats_nhoods.reset_index().sort_values(['Neighbourhood',('Household ID', 'count')], ascending=False)

In [ ]:
# save to a csv if you want
sorted_nhoods.to_csv('output_files/hof_2018_Caseload_Neighbourhood_Summary.csv')

In [ ]:
sorted_nhoods.head()

In [ ]:
# AGGREGATE BY NEIGHBOURHOOD
nhood_aggregation = households.groupby('Neighbourhood')
nh_s = nhood_aggregation.agg({'Household ID': 'count'})
nh_s.reset_index(inplace=True)
# MAKE A SERIES that maps Neighbourhood to the sum of the number of HH there
nh_hh = pd.Series(list(nh_s['Household ID']), index= list(nh_s['Neighbourhood']))

In [ ]:
# for example how mnay HH live in 'ALPINE'
nh_hh['ALPINE']

In [ ]:
nh_hh.head()

### Summarize Data by City Neighbourhood and Source of Income

In [ ]:
def lookup_total(row, index_series):
    '''
    takes a row and places the value in the Neighbourhood column
    i.e. ALPINE into a variable and then looks for it in the index_series
    which is a mapping of Neighbourhood to total number of HH in that neighbourhood 
    '''
    hood = row['Neighbourhood']
    return index_series[hood]
    
def return_city(row):
    '''
    to avoid errors resulting from bad inputs (city mismatch vs address, mis-spellings etc) that was 
    more common before the new address field formatting, we can use the fact that 
    in the neighbourhoods dataset, Kitchener Neighbourhoods are in caps and Waterloo
    are sentenance case.  This function will allow the insertion of a standardized City into a new
    column.
    
    Otherwise there are mispellings, address city mismatches and other oddities
    
    '''
    hood = row['Neighbourhood']
    if not hood:
        return 'Unknown'
    elif hood.isupper():
        return 'Kitchener'
    else:
        return 'Waterloo'

In [ ]:
# insert a new column into the data dataframe with a total HH in each Neighbourhood
data['Neighbourhood HH Total'] = data.apply(lambda row: lookup_total(row, nh_hh), axis=1)

In [ ]:
# insert the City value implied by the neighbourhood from the supplied geocoordinates
data['Implied_City'] = data.apply(lambda row: return_city(row), axis=1)

In [ ]:
# create a new sortby data object that references the new column 
hh_totals = data.groupby('Household ID').first()
hh_totals['Number of Visits'] = data['Household ID'].value_counts()
hh_totals.reset_index(inplace=True)

hh_nhood_data = hh_totals.groupby(['Neighbourhood', 'Implied_City', 'Neighbourhood HH Total', 'Household Primary Income Source'])
hh_stats_nhoods = hh_nhood_data.agg(agg_functions)
hh_sorted_nhoods = hh_stats_nhoods.reset_index()

In [ ]:
# write a csv of the summarized data
xname = 'output_files/hof_2018_EFHP_Caseload_HH_NH_summary.csv'
hh_sorted_nhoods.sort_values(['Neighbourhood HH Total',('Household ID', 'count')], ascending=False).to_csv(xname)

#### Summary stats by neighbourhood and SOI, visits and family size

In [ ]:
hh_sorted_nhoods.sort_values(['Neighbourhood HH Total',('Household ID', 'count')], ascending=False)

#### Neighbourhood Aggregate totals

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
n_hood_totals = data.sort_values('Neighbourhood HH Total', 
                                 ascending=False).drop_duplicates(subset=['Neighbourhood', 
                                                                          'Implied_City'], 
                                                                           keep='last')
n_hood_totals.reset_index(inplace=True)
sorted_nhood_totals= n_hood_totals[['Neighbourhood', 'Implied_City','Neighbourhood HH Total']]


In [ ]:
sorted_nhood_totals.to_csv('output_files/hof_2018_caseload_neighbourhood_totals.csv')